<a href="https://colab.research.google.com/github/abbyambita/Diagnosing-COVID-from-CT-Scan-Images/blob/main/trial_bagan_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/seokinj/BAGAN/

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import os 

os.chdir("/content/gdrive/My Drive")

#!ls  '/content/gdrive/My Drive/CS 284 Mini-Project/Code/'

%cd "/content/gdrive/My Drive/CS 284 Mini-Project/Code/"

/content/gdrive/.shortcut-targets-by-id/1eVFVz23F6ROX0s10Oe3tT9HVzr502iW2/CS 284 Mini-Project/Code


In [3]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import PIL
import glob
import xml.etree.ElementTree as ET
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import HTML
from torchvision.utils import save_image
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output
from scipy.stats import truncnorm
%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'

from easydict import EasyDict as edict
from PIL import Image
from collections import OrderedDict
import yaml
import imageio
import numpy as np
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import sys
import time
import math

import scipy.misc
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [4]:
workers = 6
display= 10
epoches= 100
test_epoches= 30
batch_size= 32
base_lr= 0.0002
beta1= 0.5
ano_para= 0.1
image_size= 64 
z_dim= 100
c_dim= 3
gf_dim= 64
df_dim= 64
#class_num= 10
class_num= 2
pretrained = True

dir = "epochs="+str(epoches)+"_lr="+str(base_lr)+"_batch_size="+str(batch_size)
checkpoint_dir = "model_backup/bagan/kaggle/"+dir
save_dir = "plots/bagan/kaggle/"+dir
distribution_dir ="model_result/bagan/kaggle/"+dir+"/distribution"
output_images_dir = "bagan_output_images/kaggle/"+dir

os.makedirs("model_result/bagan/kaggle/"+dir, exist_ok=True)
os.makedirs("plots/bagan/kaggle/"+dir, exist_ok=True)
os.makedirs("model_backup/bagan/kaggle/"+dir, exist_ok=True)
os.makedirs(output_images_dir, exist_ok=True)

os.makedirs("model_result/bagan/kaggle/"+dir+"/distribution", exist_ok=True)

os.makedirs(save_dir+"/accuracy", exist_ok=True)
os.makedirs(save_dir+"/loss", exist_ok=True)
os.makedirs(checkpoint_dir+"/gan", exist_ok=True)
os.makedirs(checkpoint_dir+"/vae", exist_ok=True)

In [5]:
def load_images(batch_size, image_size):
  dataroot = "revised-kaggle-validation/train"

  dataset = dset.ImageFolder(root=dataroot,
                            transform=transforms.Compose([
                                transforms.Resize(image_size),
                                transforms.RandomHorizontalFlip(),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                            ]))
  
  
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=True, num_workers=workers)

  return dataloader

In [6]:
dataloader = load_images(32, 128)
print(len(dataloader))
# real_batch = iter(dataloader).next()
# plt.figure(figsize=(15,15))
# plt.axis("off")
# plt.title("Training Images")
# image = np.transpose(vutils.make_grid(real_batch[0].to(device), normalize=True).cpu(),axes=(1,2,0))
# plt.imshow(image)

95


In [7]:
import torch
import torch.nn as nn

# Generator
class Decoder(nn.Module):
	def __init__(self, z_dim, c_dim, gf_dim):
		super(Decoder, self).__init__()

		self.convTrans0 = nn.ConvTranspose2d(z_dim, gf_dim*8, 4, 1, 0, bias=False)
		self.bn0 = nn.BatchNorm2d(gf_dim*8)
		self.relu0 = nn.ReLU(inplace=True)
		
		self.convTrans1 = nn.ConvTranspose2d(gf_dim*8, gf_dim*4, 4, 2, 1, bias=False)
		self.bn1 = nn.BatchNorm2d(gf_dim*4)
		self.relu1 = nn.ReLU(inplace=True)

		self.convTrans2 = nn.ConvTranspose2d(gf_dim*4, gf_dim*2, 4, 2, 1, bias=False)
		self.bn2 = nn.BatchNorm2d(gf_dim*2)
		self.relu2 = nn.ReLU(inplace=True)

		self.convTrans3 = nn.ConvTranspose2d(gf_dim*2, gf_dim, 4, 2, 1, bias=False)
		self.bn3 = nn.BatchNorm2d(gf_dim)
		self.relu3 = nn.ReLU(inplace=True)

		self.convTrans4 = nn.ConvTranspose2d(gf_dim, c_dim, 4, 2, 1, bias=False)
		self.tanh = nn.Tanh()

		for m in self.modules():
				if isinstance(m, nn.ConvTranspose2d):
						m.weight.data.normal_(0.0, 0.02)
						if m.bias is not None:
								m.bias.data.zero_()

	def forward(self, z):
		h0 = self.relu0(self.bn0(self.convTrans0(z)))
		h1 = self.relu1(self.bn1(self.convTrans1(h0)))
		h2 = self.relu2(self.bn2(self.convTrans2(h1)))
		h3 = self.relu3(self.bn3(self.convTrans3(h2)))
		h4 = self.convTrans4(h3)
		output = self.tanh(h4)
		return output # (c_dim, 64, 64)

# Discriminator
class Encoder(nn.Module): 
	def __init__(self, z_dim, c_dim, df_dim):
		super(Encoder, self).__init__()
		self.df_dim = df_dim

		self.conv0 = nn.Conv2d(c_dim, df_dim, 4, 2, 1, bias=False)
		self.relu0 = nn.LeakyReLU(0.2, inplace=True)
		
		self.conv1 = nn.Conv2d(df_dim, df_dim*2, 4, 2, 1, bias=False)
		self.bn1 = nn.BatchNorm2d(df_dim*2)
		self.relu1 = nn.LeakyReLU(0.2, inplace=True)

		self.conv2 = nn.Conv2d(df_dim*2, df_dim*4, 4, 2, 1, bias=False)
		self.bn2 = nn.BatchNorm2d(df_dim*4)
		self.relu2 = nn.LeakyReLU(0.2, inplace=True)

		self.conv3 = nn.Conv2d(df_dim*4, df_dim*8, 4, 2, 1, bias=False)
		self.bn3 = nn.BatchNorm2d(df_dim*8)
		self.relu3 = nn.LeakyReLU(0.2, inplace=True)

		self.fc_z1 = nn.Linear(df_dim*8*4*4, z_dim)
		self.fc_z2 = nn.Linear(df_dim*8*4*4, z_dim)

		#self.conv4 = nn.Conv2d(df_dim*8, 1, 4, 1, 0, bias=False)

		for m in self.modules():
				if isinstance(m, nn.Conv2d):
						m.weight.data.normal_(0.0, 0.02)
						if m.bias is not None:
								m.bias.data.zero_()

	def forward(self, input):
		h0 = self.relu0(self.conv0(input))
		h1 = self.relu1(self.bn1(self.conv1(h0)))
		h2 = self.relu2(self.bn2(self.conv2(h1)))
		h3 = self.relu3(self.bn3(self.conv3(h2)))
		
		mu = self.fc_z1(h3.view(-1, self.df_dim*8*4*4))	# (1, 128*8*4*4)
		sigma = self.fc_z2(h3.view(-1, self.df_dim*8*4*4))
		return mu,sigma # by squeeze, get just float not float Tenosor


class Generator(nn.Module):
        def __init__(self, z_dim, c_dim, gf_dim):
                super(Generator, self).__init__()

                self.convTrans0 = nn.ConvTranspose2d(z_dim, gf_dim*8, 4, 1, 0, bias=False)
                self.bn0 = nn.BatchNorm2d(gf_dim*8)
                self.relu0 = nn.ReLU(inplace=True)

                self.convTrans1 = nn.ConvTranspose2d(gf_dim*8, gf_dim*4, 4, 2, 1, bias=False)
                self.bn1 = nn.BatchNorm2d(gf_dim*4)
                self.relu1 = nn.ReLU(inplace=True)

                self.convTrans2 = nn.ConvTranspose2d(gf_dim*4, gf_dim*2, 4, 2, 1, bias=False)
                self.bn2 = nn.BatchNorm2d(gf_dim*2)
                self.relu2 = nn.ReLU(inplace=True)

                self.convTrans3 = nn.ConvTranspose2d(gf_dim*2, gf_dim, 4, 2, 1, bias=False)
                self.bn3 = nn.BatchNorm2d(gf_dim)
                self.relu3 = nn.ReLU(inplace=True)

                self.convTrans4 = nn.ConvTranspose2d(gf_dim, c_dim, 4, 2, 1, bias=False)
                self.tanh = nn.Tanh()


        def forward(self, z):
                h0 = self.relu0(self.bn0(self.convTrans0(z)))
                h1 = self.relu1(self.bn1(self.convTrans1(h0)))
                h2 = self.relu2(self.bn2(self.convTrans2(h1)))
                h3 = self.relu3(self.bn3(self.convTrans3(h2)))
                h4 = self.convTrans4(h3)
                output = self.tanh(h4)
                return output # (c_dim, 64, 64)

class _ganLokaggles(nn.Module):
    '''
    Layer of the GAN logits of the discriminator
    The layer gets class logits as inputs and calculates GAN logits to
    differentiate real and fake images in a numerical stable way
    '''
    def __init__(self, num_classes):
        '''
        :param num_classes: Number of real data classes (10 for SVHN)
        '''
        super(_ganLogits, self).__init__()
        self.num_classes = num_classes

    def forward(self, class_logits):
        '''
        :param class_logits: Unscaled log probabilities of house numbers
        '''

        # Set gan_logits such that P(input is real | input) = sigmoid(gan_logits).
        # Keep in mind that class_logits gives you the probability distribution over all the real
        # classes and the fake class. You need to work out how to transform this multiclass softmax
        # distribution into a binary real-vs-fake decision that can be described with a sigmoid.
        # Numerical stability is very important.
        # You'll probably need to use this numerical stability trick:
        # log sum_i exp a_i = m + log sum_i exp(a_i - m).
        # This is numerically stable when m = max_i a_i.
        # (It helps to think about what goes wrong when...
        #   1. One value of a_i is very large
        #   2. All the values of a_i are very negative
        # This trick and this value of m fix both those cases, but the naive implementation and
        # other values of m encounter various problems)
        real_class_logits, fake_class_logits = torch.split(class_logits, self.num_classes, dim=1)
        fake_class_logits = torch.squeeze(fake_class_logits)

        max_val, _ = torch.max(real_class_logits, 1, keepdim=True)
        stable_class_logits = real_class_logits - max_val
        max_val = torch.squeeze(max_val)
        gan_logits = torch.log(torch.sum(torch.exp(stable_class_logits), 1)) + max_val - fake_class_logits

        return gan_logits	# [128]

class Discriminator(nn.Module):
        def __init__(self, z_dim, c_dim, df_dim, class_num):
                super(Discriminator, self).__init__()
                self.df_dim = df_dim

                self.conv0 = nn.Conv2d(c_dim, df_dim, 4, 2, 1, bias=False)
                self.relu0 = nn.LeakyReLU(0.2, inplace=True)

                self.conv1 = nn.Conv2d(df_dim, df_dim*2, 4, 2, 1, bias=False)
                self.bn1 = nn.BatchNorm2d(df_dim*2)
                self.relu1 = nn.LeakyReLU(0.2, inplace=True)

                self.conv2 = nn.Conv2d(df_dim*2, df_dim*4, 4, 2, 1, bias=False)
                self.bn2 = nn.BatchNorm2d(df_dim*4)
                self.relu2 = nn.LeakyReLU(0.2, inplace=True)

                self.conv3 = nn.Conv2d(df_dim*4, df_dim*8, 4, 2, 1, bias=False)
                self.bn3 = nn.BatchNorm2d(df_dim*8)
                self.relu3 = nn.LeakyReLU(0.2, inplace=True)

		#self.fc_z = nn.Linear(df_dim*8*4*4, z_dim)
                self.fc_aux = nn.Linear(df_dim*8*4*4, class_num+1)
                self.softmax = nn.LogSoftmax()

                for m in self.modules():
                        if isinstance(m, nn.Linear):
                                m.weight.data.normal_(0.0, 0.02)
                                if m.bias is not None:
                                        m.bias.data.zero_()


        def forward(self, input):
                h0 = self.relu0(self.conv0(input))
                h1 = self.relu1(self.bn1(self.conv1(h0)))
                h2 = self.relu2(self.bn2(self.conv2(h1)))
                h3 = self.relu3(self.bn3(self.conv3(h2)))
                #cl = self.class_logistics(h3.view(-1, self.df_dim*8*4*4))
                #gl = self.gan_logistics(cl)    
                output = self.softmax(self.fc_aux(h3.view(-1, self.df_dim*8*4*4)))
                return h3, output
                #return h3, output.view(-1,1).squeeze(1) # by squeeze, get just float not float Tenosor

In [8]:
def conditional_latent_generator(distribution, class_num, batch):
	class_labels = torch.randint(0, class_num, (batch,), dtype=torch.long)
	fake_z = distribution[class_labels[0].item()].sample((1,))
	for c in class_labels[1:]:
		fake_z = torch.cat((fake_z, distribution[c.item()].sample((1,))), dim=0)
	return fake_z, class_labels

In [9]:
def batch2one(Z, y, z, class_num):
	for i in range(y.shape[0]):
		Z[y[i]] = torch.cat((Z[y[i]], z[i].cpu()), dim=0) # Z[label][0] should be deleted..
	return Z			
	
class AverageMeter(object):
    """ Computes ans stores the average and current value"""
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0.
        self.avg = 0.
        self.sum = 0.
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def one_hot(x, num_classes):
        '''
        One-hot encoding of the vector of classes. It uses number of classes + 1 to
        encode fake images
        :param x: vector of output classes to one-hot encode
        :return: one-hot encoded version of the input vector
        '''
        label_numpy = x.data.cpu().numpy()
        label_onehot = np.zeros((label_numpy.shape[0], num_classes + 1))
        label_onehot[np.arange(label_numpy.shape[0]), label_numpy] = 1
        return torch.FloatTensor(label_onehot)


def weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
                m.weight.data.normal_(0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)

In [10]:
def save_checkpoint(state, filename='checkpoint'):
    torch.save(state, filename + '.pth.tar')

def print_gan_log(epoch, epoches, iteration, iters, learning_rate,
              display, batch_time, data_time, D_losses, G_losses):
    print('epoch: [{}/{}] iteration: [{}/{}]\t'
          'Learning rate: {}'.format(epoch, epoches, iteration, iters, learning_rate))
    print('Time {batch_time.sum:.3f}s / {0}iters, ({batch_time.avg:.3f})\t'
          'Data load {data_time.sum:.3f}s / {0}iters, ({data_time.avg:3f})\n'
          'Loss_D = {loss_D.val:.8f} (ave = {loss_D.avg:.8f})\n'
          'Loss_G = {loss_G.val:.8f} (ave = {loss_G.avg:.8f})\n'.format(
              display, batch_time=batch_time,
              data_time=data_time, loss_D=D_losses, loss_G=G_losses))

In [11]:
#Clear
def print_vae_log(epoch, epoches, iteration, iters, learning_rate,
              display, batch_time, data_time, losses):

    print('epoch: [{}/{}] iteration: [{}/{}]\t'
          'Learning rate: {}'.format(epoch, epoches, iteration, iters, learning_rate))
    print('Time {batch_time.sum:.3f}s / {0}iters, ({batch_time.avg:.3f})\t'
          'Data load {data_time.sum:.3f}s / {0}iters, ({data_time.avg:3f})\n'
          'Loss = {loss.val:.8f} (ave = {loss.avg:.8f})\n'.format(
              display, batch_time=batch_time,
              data_time=data_time, loss=losses))

In [12]:
def plot_result2(fake, image_size, num_epoch, save_dir, name, fig_size=(8, 8), is_gray=False):

    generate_images = fake
    #G.train() # for next train after plot_result at a epoch ...

    n_rows = n_cols = 8
    fig, axes = plt.subplots(n_rows, n_cols, figsize=fig_size)

    for ax, img in zip(axes.flatten(), generate_images):
        ax.axis('off')
        ax.set_adjustable('box')
        if is_gray:
            img = img.cpu().data.view(image_size, image_size).numpy()
            ax.imshow(img, cmap='gray', aspect='equal')
        else:
            img = (((img - img.min()) * 255) / (img.max() - img.min())).cpu().data.numpy().transpose(1, 2, 0).astype(np.uint8)
            ax.imshow(img, cmap=None, aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    title = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, title, ha='center')

    if name == "dcgan":
        plt.savefig(os.path.join(save_dir, 'DCGAN_epoch_{}.png'.format(num_epoch)))
        plt.close()

    elif name == "anomaly":
        plt.savefig(os.path.join(save_dir, 'anoGAN_epoch_{}.png'.format(num_epoch)))
        plt.close()

    elif name == "vae":
        plt.savefig(os.path.join(save_dir, 'vae_epoch_{}.png'.format(num_epoch)))
        plt.close()

    elif name =="gan":
        plt.savefig(os.path.join(save_dir, 'gan_epoch_{}.png'.format(num_epoch)))
        plt.close()

def plot_result(G, fixed_noise, image_size, num_epoch, save_dir, name, fig_size=(8, 8), is_gray=False):

    G.eval()
    generate_images = G(fixed_noise)
    G.train() # for next train after plot_result at a epoch ... 
    
    n_rows = n_cols = 8
    fig, axes = plt.subplots(n_rows, n_cols, figsize=fig_size)
    
    for ax, img in zip(axes.flatten(), generate_images):
        ax.axis('off')
        ax.set_adjustable('box')
        if is_gray:
            img = img.cpu().data.view(image_size, image_size).numpy()
            ax.imshow(img, cmap='gray', aspect='equal')
        else:
            img = (((img - img.min()) * 255) / (img.max() - img.min())).cpu().data.numpy().transpose(1, 2, 0).astype(np.uint8)
            ax.imshow(img, cmap=None, aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    title = 'Epoch {0}'.format(num_epoch)
    fig.text(0.5, 0.04, title, ha='center')
    
    if name == "dcgan":
        plt.savefig(os.path.join(save_dir, 'DCGAN_epoch_{}.png'.format(num_epoch)))
        plt.close()

    elif name == "anomaly":
        plt.savefig(os.path.join(save_dir, 'anoGAN_epoch_{}.png'.format(num_epoch)))
        plt.close()

    elif name == "vae":
        plt.savefig(os.path.join(save_dir, 'vae_epoch_{}.png'.format(num_epoch)))
        plt.close()
    
    elif name =="gan":
        plt.savefig(os.path.join(save_dir, 'gan_epoch_{}.png'.format(num_epoch)))
        plt.close()

#Clear    
def plot_loss(num_epoch, epoches, save_dir, **loss):
    fig, ax = plt.subplots() 
    ax.set_xlim(0,epoches + 1)
    if len(loss) == 2:
        ax.set_ylim(0, max(np.max(loss['g_loss']), np.max(loss['d_loss'])) * 1.1)
    elif len(loss) == 1:
        ax.set_ylim(0, max(np.max(loss['vae_loss'])) * 1.1)
    plt.xlabel('Epoch {}'.format(num_epoch))
    plt.ylabel('Loss')
    
    if len(loss) == 2:
        plt.plot([i for i in range(1, num_epoch + 1)], loss['d_loss'], label='Discriminator', color='red', linewidth=3)
        plt.plot([i for i in range(1, num_epoch + 1)], loss['g_loss'], label='Generator', color='mediumblue', linewidth=3)
        plt.legend()
        plt.savefig(os.path.join(os.path.join(save_dir, "loss"), 'gan_loss_epoch_{}.png'.format(num_epoch)))
    elif len(loss) == 1:
        plt.plot([i for i in range(1, num_epoch + 1)], loss['vae_loss'], label='vae_loss', color='red', linewidht=3)
        plt.legend()
        plt.savefig(os.path.join(os.path.join(save_dir, "loss"), 'vae_loss_epoch_{}.png'.format(num_epoch)))
 
    plt.close()

#Clear
def plot_accuracy(num_epoch, epoches, save_dir, real_acc, fake_acc):
	fig, ax = plt.subplots()
	ax.set_xlim(0,epoches + 1)
	ax.set_ylim(0, max(np.max(real_acc), np.max(fake_acc)) * 1.1)
	plt.xlabel('Epoch {}'.format(num_epoch))
	plt.ylabel('Accuracy')

	plt.plot([i for i in range(1, num_epoch + 1)], real_acc, label='real data', color='red', linewidth=3)
	plt.plot([i for i in range(1, num_epoch + 1)], fake_acc, label='fake data', color='mediumblue', linewidth=3)
	plt.legend()
	plt.savefig(os.path.join(os.path.join(save_dir, "accuracy"), 'gan_accuracy_epoch_{}.png'.format(num_epoch)))

	plt.close()

In [13]:
import torch.distributions.multivariate_normal as mn

In [15]:
#run as pretrained = False first then run again with pretrained = True??? Baket?
pretrained = True
if __name__ == "__main__":
  use_cuda = torch.cuda.is_available()
  gpu = 0

  train_loader = load_images(batch_size, image_size)


  decoder = Decoder(z_dim, c_dim, gf_dim)
  encoder = Encoder(z_dim, c_dim, df_dim)

  if not pretrained:
    if use_cuda:
      decoder = decoder.cuda(gpu)
      encoder = encoder.cuda(gpu)

    # WHY BECLoss() - only need to determine fake/real for Discriminator
    criterion = nn.BCELoss()
    if use_cuda:
      criterion = criterion.cuda(gpu)

    optimizerE = torch.optim.Adam(encoder.parameters(), lr=base_lr, betas=(beta1, 0.999))
    optimizerD = torch.optim.Adam(decoder.parameters(), lr=base_lr, betas=(beta1, 0.999))

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    fixed_noise = torch.FloatTensor(8 * 8, z_dim, 1, 1).normal_(0, 1)
    if use_cuda:
      fixed_noise = fixed_noise.cuda(gpu)
    with torch.no_grad():
      fixed_noisev = fixed_noise

    end = time.time()
    
    encoder.train()
    decoder.train()
    loss_list = []

    
    criterion = nn.MSELoss(size_average=False)	
    for epoch in range(epoches):
      for i, (input, label) in enumerate(train_loader):
        #l Update 'D' : max log(D(x)) + log(1-D(G(z)))
        data_time.update(time.time()-end)
      
        batch_size = input.size(0)
        if use_cuda:
          input = input.cuda(gpu)
        
        mu, log_sigmoid = encoder(input)
        # reparameterization
        std = torch.exp(log_sigmoid/2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        z = z.view(-1, z_dim, 1, 1)	
        if use_cuda:
          z = z.cuda(gpu)

        # reconstruct image
        x_reconstruct = decoder(z)

        # reconstruct_loss + KL_divergence
        reconstruct_loss = criterion(x_reconstruct, input)
        kl_div = -0.5 * torch.sum(1+log_sigmoid-mu.pow(2)-log_sigmoid.exp())
        loss = reconstruct_loss + kl_div
        losses.update(loss.item())	
        optimizerE.zero_grad()
        optimizerD.zero_grad()
        loss.backward()
        optimizerE.step()
        optimizerD.step()

        batch_time.update(time.time()-end)
        end = time.time()
    
        # log every 100th train data of train_loader - display(100)	
        if (i+1) % display == 0:
          print_vae_log(epoch+1, epoches, i+1, len(train_loader), base_lr, display, batch_time, data_time, losses)
          # Is it Continous ???
          batch_time.reset()
          data_time.reset()
        # log every 1 epoch (all of train_loader)
        elif (i+1) == len(train_loader):
          print_vae_log(epoch + 1, epoches, i + 1, len(train_loader), base_lr, (i + 1) % display, batch_time, data_time, losses)
          batch_time.reset()
          data_time.reset()

      # log every 1 epoch
      loss_list.append(losses.avg)
      losses.reset()

      plot_result(decoder, fixed_noisev, image_size, epoch + 1,save_dir, 'vae', is_gray=(c_dim == 1))
      #plot_loss(epoch+1, config.epoches, args.save_dir, vae_loss=loss_list)
      # save the D and G.
      save_checkpoint({'epoch': epoch, 'state_dict': encoder.state_dict(),}, os.path.join(os.path.join(checkpoint_dir,"vae"), 'encoder_epoch_{}'.format(epoch)))
      save_checkpoint({'epoch': epoch, 'state_dict': decoder.state_dict(),}, os.path.join(os.path.join(checkpoint_dir,"vae"), 'decoder_epoch_{}'.format(epoch)))

    #create_gif(epoches, save_dir, 'vae')

  ## Class Conditional Generator - Pretrained Model"
  else:
    print("Class Conditional Generator - Use Pretrained Model")
    if use_cuda:
      encoder = encoder.cuda(gpu)
      decoder = decoder.cuda(gpu)
    encoder.load_state_dict(torch.load(os.path.join(os.path.join(checkpoint_dir, "vae"), "encoder_epoch_"+ str(epoches-1) + ".pth.tar"))['state_dict'])
    decoder.load_state_dict(torch.load(os.path.join(os.path.join(checkpoint_dir, "vae"), "decoder_epoch_"+ str(epoches-1) + ".pth.tar"))['state_dict'])
    #Z = np.empty([config.class_num, config.z_dim], dtype=float)
    # Z : [label-1, labe-2, ... ]
    # Z[label-1] : [[z1], [z2], ... ] (#labeld_data, #z_dim)
    encoder.eval()
    decoder.eval()
    Z = []
    with torch.no_grad():
      for i in range(class_num):
        Z.append(torch.zeros((1, z_dim), dtype=torch.float)) # Z : [class_num, z_dim]
    
      for i, (input, label) in enumerate(train_loader):
        if use_cuda:
          input = input.cuda(gpu)
        mu, log_sigmoid = encoder(input)
        std = torch.exp(log_sigmoid/2)
        eps = torch.randn_like(std)
        z = mu + eps * std
        z = z.view(-1, 1, z_dim)
        Z = batch2one(Z, label, z, class_num)

      N = []
      for i in range(class_num):
        label_mean = torch.mean(Z[i][1:], dim=0).float()
        label_cov = torch.from_numpy(np.cov(Z[i][1:].numpy(), rowvar=False)).float()
        #print("{}th Z : {}".format(i+1, Z[i][1:].shape))
        #print("{}-class  mean : {}".format(i+1, label_mean.shape))
        #print("{}-class covariance : {}".format(i+1, label_cov.shape))
        m = mn.MultivariateNormal(label_mean, label_cov)
        sample = m.sample((64,))
        print(sample.shape)
        if use_cuda:
          sample = sample.cuda(gpu)
        fake = decoder(sample.view(-1, z_dim, 1, 1))
        
        
        plot_result2(fake, image_size, i, 'data/gan/samples', 'ssgan', is_gray=(c_dim == 1))
        N.append(m)
      print("uhm")
      torch.save({'distribution': N}, os.path.join(distribution_dir, 'class_distribution')+'.dt')

Class Conditional Generator - Use Pretrained Model
torch.Size([64, 100])
torch.Size([64, 100])
uhm


In [ ]:
pretrained = True
if __name__ == "__main__":
  use_cuda = torch.cuda.is_available()
  gpu = 0

  # parser.add_argument('--save_dir', type=str, dest='save_dir', help='the path of generated data dir', default='sample')
  # parser.add_argument('--distribution_dir', type=str, dest='distribution_dir', help='the path of class distribution dir', default='distribution')
  # parser.add_argument('--test_dir', type=str, dest='test_dir', help='the path of anomaly test data')
  # parser.add_argument('--test_result_dir', type=str, dest='test_result_dir', help='the path of anomaly test result dir')
  train_loader = load_images(batch_size, image_size)

  G = Generator(z_dim, c_dim, gf_dim)
  G.apply(weights_init)

  D = Discriminator(z_dim, c_dim, df_dim, class_num)
  D.apply(weights_init)

  # if not pretrained:
  #   if use_cuda:
  #     print("duh")
  #     G = G.cuda(gpu)
  #     D = D.cuda(gpu)

  if use_cuda:
    print("duh")
    G = G.cuda(gpu)
    D = D.cuda(gpu)


  distribution = torch.load(os.path.join(distribution_dir,'class_distribution.dt'))['distribution']

  #G.load_state_dict(torch.load(os.path.join(os.path.join(args.checkpoint_dir,"vae") , "decoder_epoch_"+str(config.epoches-1) + ".pth.tar"))['state_dict'])

  # state_e = torch.load(os.path.join(os.path.join(args.checkpoint_dir,"vae"), "encoder_epoch_"+str(config.epoches-1) + ".pth.tar"))['state_dict']
  # del state_e['fc_z1.weight']
  # del state_e['fc_z1.bias']
  # del state_e['fc_z2.weight']
  # del state_e['fc_z2.bias']
  # state_e.update({'fc_aux.weight':D.state_dict()['fc_aux.weight']})
  # state_e.update({'fc_aux.bias':D.state_dict()['fc_aux.bias']})

  # D.load_state_dict(state_e)

  criterion = nn.NLLLoss()

  optimizerD = torch.optim.Adam(D.parameters(), lr=base_lr, betas=(beta1, 0.999))
  optimizerG = torch.optim.Adam(G.parameters(), lr=base_lr, betas=(beta1, 0.999))

  batch_time = AverageMeter()
  data_time = AverageMeter()
  D_losses = AverageMeter()
  G_losses = AverageMeter()

  #fixed_noise = torch.FloatTensor(8 * 8, config.z_dim, 1, 1).normal_(0, 1)
  #if use_cuda:
  #	fixed_noise = fixed_noise.cuda(gpu)
  #with torch.no_grad():
  #	fixed_noisev = fixed_noise

  end = time.time()

  D.train()
  G.train()
  D_loss_list = []
  G_loss_list = []

  real_label = torch.LongTensor(batch_size)
  fake_label = torch.LongTensor(batch_size)	

  for epoch in range(epoches):
    total_real = 0
    total_fake = 0
    correct_real = 0
    correct_fake = 0
    for i, (input, label) in enumerate(train_loader):
      # Update 'D' : max log(D(x)) + log(1-D(G(z)))
      data_time.update(time.time()-end)
      batch_size = input.size(0)
      fake_num = math.ceil(batch_size/class_num)	# For each batch, 1/(n+1) of total images are fake
      conditional_z, z_label = conditional_latent_generator(distribution, class_num, batch_size)

      label = label.long().squeeze() # "squeeze" : [batch, 1] --> [batch] ... e.g) [1,2,3,4...]		
      #print(label[0])
      #print(path[0])
      if use_cuda:
        input = input.cuda(gpu)
        label = label.cuda(gpu)

      sample_features, D_real = D(input)
      real_label.resize_(batch_size).copy_(label)	# "cpu" : gpu --> cpu // <<.data.cpu vs cpu>> // "resize_as" : get tensor size and resize 
      if use_cuda:
        real_label = real_label.cuda(gpu) 
      
      D_loss_real = criterion(D_real, real_label)
      noise = conditional_z[0:fake_num].view(-1, z_dim, 1, 1)

      fake_label.resize_(noise.shape[0]).fill_(class_num)	# fake_label = '(num_class)+1'
      if use_cuda:
        noise = noise.cuda(gpu)
        fake_label = fake_label.cuda(gpu)
        z_label = z_label.cuda(gpu)
      
      fake = G(noise)

      _, D_fake = D(fake.detach())	# Fake image...
      D_loss_fake = criterion(D_fake, fake_label)	# Hmmmm...... fake_label? or z_label?

      D_loss = D_loss_real + D_loss_fake
      D_losses.update(D_loss.item())
      D.zero_grad()
      G.zero_grad()
      D_loss.backward()
      optimizerD.step()

      # Update 'G' : max log(D(G(z)))
      noise = conditional_z.view(-1, z_dim, 1, 1)
      if use_cuda:
        noise = noise.cuda(gpu)
      fake = G(noise)
      _, D_fake = D(fake)
      G_loss = criterion(D_fake, z_label)
      #G_losses.update(G_loss.data[0])
      G_losses.update(G_loss.item())

      D.zero_grad()
      G.zero_grad()
      G_loss.backward()
      optimizerG.step()

      batch_time.update(time.time()-end)
      end = time.time()
      
      pred_real = torch.max(D_real.data, 1)[1]
      pred_fake = torch.max(D_fake.data, 1)[1]
      total_real += real_label.size(0)
      total_fake += z_label.size(0)
      correct_real += (pred_real == real_label).sum().item()
      correct_fake += (pred_fake == z_label).sum().item()

      # log every 100th train data of train_loader - display(100)	
      if (i+1) % display == 0:
        print_gan_log(epoch+1, epoches, i+1, len(train_loader), base_lr, display, batch_time, data_time, D_losses, G_losses)
        # Is it Continous ???
        batch_time.reset()
        data_time.reset()
      # log every 1 epoch (all of train_loader) ... "End of all mini-Batch"
      elif (i+1) == len(train_loader):
        print_gan_log(epoch + 1, epoches, i + 1, len(train_loader), base_lr,
                          (i + 1) % display, batch_time, data_time, D_losses, G_losses)
        #accuracy = masked_correct.item()/max(1.0, num_samples.item())
        plot_result2(fake, image_size, epoch + 1, save_dir, 'gan', is_gray=(c_dim == 1))
        real_acc = 100 * correct_real / total_real
        fake_acc = 100 * correct_fake / total_fake
        print('Real Accuracy : {}'.format(real_acc))
        print('Fake Accuracy : {}'.format(fake_acc))
        #plot_accuracy(epoch+1, epoches, save_dir, real_acc, fake_acc)
        batch_time.reset()
        data_time.reset()

    # log every 1 epoch
    D_loss_list.append(D_losses.avg)
    G_loss_list.append(G_losses.avg)
    D_losses.reset()
    G_losses.reset()

    plot_result(G, fixed_noisev, image_size, epoch + 1, save_dir, 'gan', is_gray=(c_dim == 1))
    plot_loss(epoch+1,epoches, save_dir, d_loss=D_loss_list, g_loss=G_loss_list)
    # save the D and G.
    #save_checkpoint({'epoch': epoch, 'state_dict': D.state_dict(),}, os.path.join(os.path.join(checkpoint_dir,'gan'), 'D_epoch_{}'.format(epoch)))
    #save_checkpoint({'epoch': epoch, 'state_dict': G.state_dict(),}, os.path.join(os.path.join(checkpoint_dir,'gan'), 'G_epoch_{}'.format(epoch)))

    torch.save(D, os.path.join(os.path.join(checkpoint_dir,'gan'), 'D_epoch_{}.pth'.format(epoch)))
    torch.save(G, os.path.join(os.path.join(checkpoint_dir,'gan'), 'G_epoch_{}.pth'.format(epoch)))

duh


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:198: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: [1/100] iteration: [10/233]	Learning rate: 0.0002
Time 1.359s / 10iters, (0.136)	Data load 0.644s / 10iters, (0.064433)
Loss_D = 1.55756938 (ave = 1.94280602)
Loss_G = 8.60314655 (ave = 7.80654635)

epoch: [1/100] iteration: [20/233]	Learning rate: 0.0002
Time 0.673s / 10iters, (0.067)	Data load 0.086s / 10iters, (0.008557)
Loss_D = 0.96634412 (ave = 1.70442106)
Loss_G = 8.30878067 (ave = 8.24787543)

epoch: [1/100] iteration: [30/233]	Learning rate: 0.0002
Time 0.645s / 10iters, (0.064)	Data load 0.066s / 10iters, (0.006633)
Loss_D = 3.14097261 (ave = 1.87788146)
Loss_G = 10.09541702 (ave = 9.10091146)

epoch: [1/100] iteration: [40/233]	Learning rate: 0.0002
Time 0.602s / 10iters, (0.060)	Data load 0.085s / 10iters, (0.008482)
Loss_D = 1.41279531 (ave = 1.74970966)
Loss_G = 15.11264038 (ave = 9.71687341)

epoch: [1/100] iteration: [50/233]	Learning rate: 0.0002
Time 0.576s / 10iters, (0.058)	Data load 0.085s / 10iters, (0.008501)
Loss_D = 0.76795095 (ave = 1.64930639)
Loss_G =

In [ ]:
#load the saved models here
def generate_images(batch_size):
  #fake_num = math.ceil(batch_size/class_num)
  conditional_z, z_label = conditional_latent_generator(distribution, 2, batch_size)

  noise = conditional_z.view(-1, z_dim, 1, 1)
  print(noise.shape)
  if use_cuda:
    noise = noise.cuda(gpu)

  fake = G(noise)
  print(fake.shape)

  for i, label in enumerate(z_label):
    cv = output_images_dir+"/covid"
    ncv=output_images_dir+"/noncovid"

    os.makedirs(cv, exist_ok = True)
    os.makedirs(ncv, exist_ok = True)

    if label == 0:
      vutils.save_image(fake[i].detach(), ncv+'/fake_samples_{:03d}.png'.format(i),normalize=True, nrow=5)
    else:
      vutils.save_image(fake[i].detach(), cv+'/fake_samples_{:03d}.png'.format(i),normalize=True, nrow=5)


generate_images(2000)

torch.Size([2000, 100, 1, 1])
torch.Size([2000, 3, 64, 64])


In [ ]:
# G = torch.load(checkpoint_dir+"/gan/G_epoch_19.pth")
# generate_images(50)

torch.Size([32, 100, 1, 1])
torch.Size([32, 3, 64, 64])
